In [72]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
import numpy as np
from numpy import linalg
import scipy.io as spio
import matplotlib.pyplot as plt
from matplotlib import pyplot


In [74]:
def confusionMatrix(y_actual, y_predicted):
    tp = 0                                        #                   Altual class             
    tn = 0                                        #                 ____1______0______
    fp = 0                                        #Predicted    1  |   TP   |   FP   |
    fn = 0                                        # class       ---------------------
                                                  #             0  |___FN___|___TN__|
    for i in range(len(y_actual)):
        if y_actual[i] == 1:
            if y_actual[i] == y_predicted[i]:
                tp = tp + 1
            else:
                fn = fn + 1
        if y_actual[i] == 0:
            if y_actual[i] == y_predicted[i]:
                tn = tn + 1
            else:
                fp = fp + 1
                
    cm = [[tn, fp], [fn, tp]]
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    sens = tp/(tp+fn)
    prec = tp/(tp+fp)
    fm = (2*prec*sens)/(prec+sens)
    return cm, accuracy, fm

In [75]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)
    
def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    numerator = np.linalg.norm(x-y)**2
    denominator = 2 * (sigma ** 2)
    return np.exp(-numerator / denominator)

In [76]:
class SVM(object):

    def __init__(self, kernel=linear_kernel, tol=1e-3, C=0.1,
                 max_passes=5, sigma=0.1):

        self.kernel = kernel
        self.tol = tol
        self.C = C
        self.max_passes = max_passes
        self.sigma = sigma
        self.model = dict()

    def __repr__(self):
        return (f"{self.__class__.__name__}("
                f"kernel={self.kernel.__name__}, "
                f"tol={self.tol}, "
                f"C={self.C}, "
                f"max_passes={self.max_passes}, "
                f"sigma={self.sigma}"
                ")")

    def svmTrain(self, X, Y):
        # Data parameters
        m = X.shape[0]

        # Map 0 to -1
        Y = np.where(Y == 0, -1, 1)

        # Variables
        alphas = np.zeros((m, 1), dtype=float)
        b = 0.0
        E = np.zeros((m, 1), dtype=float)
        passes = 0

        # Pre-compute the kernel matrix
        if self.kernel.__name__ == 'linear_kernel':
            print(f'Pre-computing {self.kernel.__name__} kernel matrix')
            K = X @ X.T

        elif self.kernel.__name__ == 'gaussian_kernel':
            print(f'Pre-computing {self.kernel.__name__} kernel matrix')
            X2 = np.sum(np.power(X, 2), axis=1).reshape(-1, 1)
            K = X2 + (X2.T - (2 * (X @ X.T)))
            K = np.power(self.kernel(1, 0, self.sigma), K)

        else:
            # Pre-compute the Kernel Matrix
            # The following can be slow due to lack of vectorization
            print(f'Pre-computing {self.kernel.__name__} kernel matrix')
            K = np.zeros((m, m))

            for i in range(m):
                for j in range(m):
                    x1 = np.transpose(X[i, :])
                    x2 = np.transpose(X[j, :])
                    K[i, j] = self.kernel(x1, x2)
                    K[i, j] = K[j, i]

        print('Training...')
        print('This may take 1 to 2 minutes')

        while passes < self.max_passes:
            num_changed_alphas = 0

            for i in range(m):
                # Calculate Ei = f(x(i)) - y(i) using (2). 
                # E(i) = b + sum (X(i, :) * (repmat(alphas.*Y,1,n).*X)') - Y(i);
                E[i] = b + np.sum(alphas * Y * K[:, i].reshape(-1, 1)) - Y[i]

                if (Y[i] * E[i] < -self.tol and alphas[i] < self.C) or (Y[i] * E[i] > self.tol and alphas[i] > 0):
                    # In practice, there are many heuristics one can use to select
                    # the i and j. In this simplified code, we select them randomly.
                    j = np.random.randint(0, m)
                    while j == i:
                        # make sure i is not equal to j
                        j = np.random.randint(0, m)
                    
                    # Calculate Ej = f(x(j)) - y(j) using (2).
                    E[j] = b + np.sum(alphas * Y *
                                      K[:, j].reshape(-1, 1)) - Y[j]

                    # Save old alphas
                    alpha_i_old = alphas[i, 0]
                    alpha_j_old = alphas[j, 0]

                    # Compute L and H by (10) or (11)
                    if Y[i] == Y[j]:
                        L = max(0, alphas[j] + alphas[i] - self.C)
                        H = min(self.C, alphas[j] + alphas[i])
                    else:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(self.C, self.C + alphas[j] - alphas[i])
                    if L == H:
                        # continue to next i
                        continue

                    # compute eta by (14)
                    eta = 2 * K[i, j] - K[i, i] - K[j, j]
                    if eta >= 0:
                        # continue to next i
                        continue

                    # compute and clip new value for alpha j using (12) and (15)
                    alphas[j] = alphas[j] - (Y[j] * (E[i] - E[j])) / eta

                    # Clip
                    alphas[j] = min(H, alphas[j])
                    alphas[j] = max(L, alphas[j])

                    # Check if change in alpha is significant
                    if np.abs(alphas[j] - alpha_j_old) < self.tol:
                        # continue to the next i
                        # replace anyway
                        alphas[j] = alpha_j_old
                        continue

                    # Determine value for alpha i using (16)
                    alphas[i] = alphas[i] + Y[i] * \
                        Y[j] * (alpha_j_old - alphas[j])

                    # Compute b1 and b2 using (17) and (18) respectively.
                    b1 = b - E[i] - Y[i] * (alphas[i] - alpha_i_old) * \
                        K[i, j] - Y[j] * (alphas[j] - alpha_j_old) * K[i, j]

                    b2 = b - E[j] - Y[i] * (alphas[i] - alpha_i_old) * \
                        K[i, j] - Y[j] * (alphas[j] - alpha_j_old) * K[j, j]

                    # Compute b by (19).
                    if 0 < alphas[i] < self.C:
                        b = b1
                    elif 0 < alphas[j] < self.C:
                        b = b2
                    else:
                        b = (b1 + b2) / 2
                    num_changed_alphas = num_changed_alphas + 1

            if num_changed_alphas == 0:
                passes = passes + 1
            else:
                passes = 0

            print('.', end='', flush=True)

        print('\n DONE! ')

        # Save the model
        idx = alphas > 0
        self.model['X'] = X[idx.reshape(1, -1)[0], :]
        self.model['y'] = Y[idx.reshape(1, -1)[0]]
        self.model['kernelFunction'] = self.kernel
        self.model['b'] = b
        self.model['alphas'] = alphas[idx.reshape(1, -1)[0]]
        self.model['w'] = np.transpose(np.matmul(np.transpose(alphas * Y), X))
        # return model


    def predict(self, X):
        if X.shape[1] == 1:
            X = np.transpose(X)

        # Dataset
        m = X.shape[0]
        p = np.zeros((m, 1))
        pred = np.zeros((m, 1))

        if self.model['kernelFunction'].__name__ == 'linear_kernel':
            p = X.dot(self.model['w']) + self.model['b']

        elif self.model['kernelFunction'].__name__ == 'gaussian_kernel':
            # Vectorized RBF Kernel
            # This is equivalent to computing the kernel
            # on every pair of examples
            X1 = np.sum(np.power(X, 2), axis=1).reshape(-1, 1)
            X2 = np.transpose(np.sum(np.power(self.model['X'], 2), axis=1))
            K = X1 + (X2.T - (2 * (X @ (self.model['X']).T)))
            K = np.power(self.model['kernelFunction'](1, 0, self.sigma), K)
            K = np.transpose(self.model['y']) * K
            K = np.transpose(self.model['alphas']) * K
            p = np.sum(K, axis=1)

        else:
            for i in range(m):
                prediction = 0
                for j in range(self.model['X'].shape[0]):
                    prediction = prediction + self.model['alphas'][j] \
                        * self.model['y'][j] * \
                        self.model['kernelFunction'](np.transpose(
                            X[i, :]), np.transpose(self.model['X'][j, :]))

                p[i] = prediction + self.model['b']

        # Convert predictions into 0 and 1
        pred[p >= 0] = 1
        return pred

In [77]:
import os
DATA_DIR = 'dataset'

train = spio.loadmat(os.path.join(DATA_DIR, 'spamTrain.mat'))
test = spio.loadmat(os.path.join(DATA_DIR, 'spamTest.mat'))
data = spio.loadmat(os.path.join(DATA_DIR, 'ex6data1.mat'))

In [78]:
X, y = data['X'], data['y'][:, 0]

X_train = np.double(train.get('X'))
y_train = np.double(train.get('y'))
X_test = np.double(test.get('Xtest'))
y_test = np.double(test.get('ytest'))

In [79]:
X_train.shape
y_train.shape


(4000, 1)

In [80]:
model = SVM()
model

SVM(kernel=linear_kernel, tol=0.001, C=0.1, max_passes=5, sigma=0.1)

In [63]:
# %%time
model.svmTrain(X_train, y_train)


Pre-computing linear_kernel kernel matrix
Training...
This may take 1 to 2 minutes
........................................................................................................................................................................................................................................................................................................
 DONE! 


In [64]:
y_predicted = model.predict(X_train)
cm, acc, fm = confusionMatrix(y_train, y_predicted)
acc, fm


(0.99825, 0.9972581276929103)

In [65]:
cm

[[2720, 3], [4, 1273]]

In [66]:
y_predicted = model.predict(X_test)
cm, acc, fm = confusionMatrix(y_test, y_predicted)
print('Accuracy --> ', acc)
print('Confusion Matrix --> ', cm)

Accuracy -->  0.988
Confusion Matrix -->  [[683, 9], [3, 305]]


In [67]:
def visualizeBoundaryLinear(X, y, model):
    """
    Plots a linear decision boundary learned by the SVM.
    Parameters
    ----------
    X : array_like
        (m x 2) The training data with two features (to plot in a 2-D plane).
    y : array_like
        (m, ) The data labels.
    model : dict
        Dictionary of model variables learned by SVM.
    """
    w, b = model['w'], model['b']
    xp = np.linspace(min(X[:, 0]), max(X[:, 0]), 100)
    yp = -(w[0] * xp + b)/w[1]

    plotData(X, y)
    pyplot.plot(xp, yp, '-b')


In [68]:

def plotData(X, y, grid=False):
    """
    Plots the data points X and y into a new figure. Uses `+` for positive examples, and `o` for
    negative examples. `X` is assumed to be a Mx2 matrix
    Parameters
    ----------
    X : numpy ndarray
        X is assumed to be a Mx2 matrix.
    y : numpy ndarray
        The data labels.
    grid : bool (Optional)
        Specify whether or not to show the grid in the plot. It is False by default.
    Notes
    -----
    This was slightly modified such that it expects y=1 or y=0.
    """
    # Find Indices of Positive and Negative Examples
    pos = y == 1
    neg = y == 0

    # Plot Examples
    pyplot.plot(X[pos, 0], X[pos, 1], 'X', mew=1, ms=10, mec='k')
    pyplot.plot(X[neg, 0], X[neg, 1], 'o', mew=1, mfc='y', ms=10, mec='k')
    pyplot.grid(grid)


In [69]:

def visualizeBoundary(X, y, model):
    """
    Plots a non-linear decision boundary learned by the SVM and overlays the data on it.
    Parameters
    ----------
    X : array_like
        (m x 2) The training data with two features (to plot in a 2-D plane).
    y : array_like
        (m, ) The data labels.
    model : dict
        Dictionary of model variables learned by SVM.
    """
    plotData(X, y)

    # make classification predictions over a grid of values
    x1plot = np.linspace(min(X[:, 0]), max(X[:, 0]), 100)
    x2plot = np.linspace(min(X[:, 1]), max(X[:, 1]), 100)
    X1, X2 = np.meshgrid(x1plot, x2plot)

    vals = np.zeros(X1.shape)
    for i in range(X1.shape[1]):
        this_X = np.stack((X1[:, i], X2[:, i]), axis=1)
        vals[:, i] = svmPredict(model, this_X)

    pyplot.contour(X1, X2, vals, colors='y', linewidths=2)
    pyplot.pcolormesh(X1, X2, vals, cmap='YlGnBu', alpha=0.25, edgecolors='None', lw=0)
    pyplot.grid(False)


In [85]:
C = 1
tol=1e-3
max_passes=20
model = SVM( linear_kernel, tol, C, max_passes)

model.svmTrain(X, y)
visualizeBoundaryLinear(X, y, model.model)


Pre-computing linear_kernel kernel matrix
Training...
This may take 1 to 2 minutes
....................................................
 DONE! 


ValueError: operands could not be broadcast together with shapes (51,) (100,) 